# Predictive Maintenance

In [1]:
# Predictive Maintenance
from sklearn.linear_model import LogisticRegression
from rpy2 import robjects
import pandas as pd
import numpy as np

## rpy2

In [2]:
from rpy2.robjects import pandas2ri
from rpy2.robjects import default_converter
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.lib.dplyr import DataFrame

C:\Users\Kajornphat\Anaconda3\lib\site-packages\rpy2\robjects\lib\dplyr.py:24: UserWarning: This was designed againt dplyr version 0.7.5 but you have 0.8.3
  warnings.warn('This was designed againt dplyr version %s but you have %s' % (TARGET_VERSION, dplyr.__version__))


In [3]:
def convert_df(dataframe):      
    '''Convert between 
    Pandas dataframe to R "Dataframe"'''
    with localconverter(default_converter + pandas2ri.converter) as cv:
        try:
                dataframe = pandas2ri.ri2py(dataframe)
        except:
                dataframe = DataFrame(dataframe)
    return dataframe

In [4]:
r = robjects.r

### Data Preperation

In [5]:
file_name = ["cement_mill", "roller_press", "separator", "feeder"]
df = {file:pd.read_csv('Data/' + file + '.csv') for file in file_name}

In [6]:
df1 = pd.merge(df["cement_mill"], df["roller_press"], on='t_stamp')
df2 = pd.merge(df["separator"], df["feeder"], on='t_stamp')
df = pd.merge(df1, df2, on='t_stamp')

#### Remove SV and MV Features

In [7]:
### Remove SV and MV Features
df = df[df.columns[~df.columns.str.endswith('sv')]]
df = df[df.columns[~df.columns.str.endswith('mv')]]

### R Scripts 
1. Replace 0 value as NA
2. Remove Columns with Low Variance
3. Label Target by Timestamp (1 Month before Breakdown)

In [8]:
r.assign('dataset', convert_df(df))
r('''

library(caret)
library(dplyr)

bad_cols <- nearZeroVar(dataset)
dataset <- dataset[, -bad_cols]

dataset[dataset == 0] <- NA

dataset["target"] <- ifelse(dataset$t_stamp >= "2017-12-07", 1, 0)

dataset <- dataset %>%
    filter(get("mill drive_power") > 4000)

''')
df = convert_df(r('dataset'))

RRuntimeError: Error in get("mill drive_power") : object 'mill drive_power' not found


In [9]:
df

,t_stamp,be.mill_out_amp,mill.drive_power,mill_ch1_water.spray,mill_in_pres,mill_out_pres,mill_out_temp,be.rp_in_amp,feed.gate1_open,feed.gate2_open,...,rp.drive.m2_power,slide.gate_drive_open,slide.gate_nondrive_open,sep.fan_damper,sep.fan_power,sep_motor_amp,sep_motor_speed,hopper_cli_lev,tot_feed,target
0,2017-07-01 00:00:00.000,256.712189,4289.811523,0.052340,-17.973175,-249.635437,123.146561,125.709282,63.688377,99.281677,...,NaN,NaN,NaN,99.679148,4.697997e+02,383.394409,933.053955,13.489513,294.706024,0.0
1,2017-07-01 00:10:00.000,254.321503,4276.529297,0.052341,-19.417152,-247.502518,123.246750,117.173439,63.789902,99.315674,...,NaN,NaN,NaN,99.677608,4.696246e+02,384.133240,933.029663,15.617309,282.521729,0.0
2,2017-07-01 00:20:00.000,255.317444,4326.605957,0.052342,-19.352036,-247.159821,123.486923,110.041626,63.746361,99.275795,...,NaN,NaN,NaN,99.676067,4.700118e+02,388.665985,932.925537,14.310499,271.610840,0.0
3,2017-07-01 00:30:00.000,251.911148,4277.915039,0.052342,-19.673306,-248.683411,123.765419,123.561905,63.767574,99.303406,...,NaN,NaN,NaN,99.674527,4.713268e+02,385.120728,933.005127,12.195548,289.420166,0.0
4,2017-07-01 00:40:00.000,246.957001,4254.054688,0.052343,-23.307371,-242.536255,124.082153,116.652908,63.762009,99.278961,...,NaN,NaN,NaN,99.673691,4.701608e+02,382.024109,933.044678,13.203295,286.063690,0.0
5,2017-07-01 00:50:00.000,255.463150,4274.890625,0.052344,-21.198851,-246.369476,124.522644,119.447693,63.781708,99.322998,...,NaN,NaN,NaN,99.673151,4.653141e+02,376.000000,933.159424,14.806042,264.079437,0.0
6,2017-07-01 01:00:00.000,251.720367,4294.589355,0.052345,-20.949734,-248.766068,125.375626,124.633209,63.703827,99.287384,...,NaN,NaN,NaN,99.672781,4.662546e+02,375.038513,933.830566,12.283951,274.842102,0.0
7,2017-07-01 01:10:00.000,243.284653,4301.672852,0.052345,-21.743502,-247.775116,126.403473,122.064980,63.780724,99.310478,...,NaN,NaN,NaN,99.672411,4.671475e+02,372.841919,933.021240,12.668695,280.989471,0.0
8,2017-07-01 01:20:00.000,251.459930,4274.218262,0.052346,-20.453115,-246.551163,127.258850,131.661072,63.847057,99.289261,...,NaN,NaN,NaN,99.672041,4.669489e+02,375.130829,932.928528,13.118477,298.938812,0.0
9,2017-07-01 01:30:00.000,246.580521,4302.017090,0.052347,-20.030378,-244.412094,128.034943,125.936584,63.831783,99.279907,...,NaN,NaN,NaN,99.671671,4.675049e+02,369.688934,930.864502,14.115992,284.501099,0.0
